In [ ]:
#! pip install -q -U google-genai

In [21]:
#! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() # Load environment variables from .env file

# Control LangChain tracing to LangSmith / LangChain Cloud
# The intent here is to disable automatic tracing by default to avoid 403/Forbidden errors
# Set to 'false' to disable tracing. Set to 'true' and provide a valid
# LANGCHAIN_ENDPOINT and LANGCHAIN_API_KEY to enable tracing to LangChain Cloud.
os.environ['LANGCHAIN_TRACING_V2'] = 'false'  # disable automatic POSTs to LangSmith/LangChain Cloud
# Remove any LANGCHAIN_ENDPOINT that would send runs to LangSmith by default (if present)
os.environ.pop('LANGCHAIN_ENDPOINT', None)

# Keep project/api-key variables (if you later enable tracing) and OpenAI key
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT_DEMO")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [ ]:
# from google import genai

# # The client gets the API key from the environment variable `GEMINI_API_KEY`.
# client = genai.Client()

# response = client.models.generate_content(
#     model="gemini-2.5-flash", contents="Explain how AI works in a few words"
# )
# print(response.text)

AI learns patterns from data to make smart decisions or predictions.


In [ ]:
# from openai import OpenAI

# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key=os.getenv("OPENROUTER_API_KEY"),
# )

# completion = client.chat.completions.create(
#   extra_headers={
#     "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
#     "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
#   },
#   extra_body={},
#   model="openai/gpt-oss-20b:free",
#   messages=[
#     {
#       "role": "user",
#       "content": "What is the meaning of life?"
#     }
#   ]
# )
# print(completion.choices[0].message.content)

Part 2 : Indexing

In [7]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [22]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [23]:
#OpenAI
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1536

In [24]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8806977856520077


In [ ]:
# Gemini API
#  result_question = client.models.embed_content(
#         model="gemini-embedding-001",
#         contents=question)

# result_doc = client.models.embed_content(
#         model="gemini-embedding-001",
#         contents=document)

# print("result_question =", result_question.embeddings)
# print("result_doc =", result_doc.embeddings)

result_question = [ContentEmbedding(
  values=[
    0.008137716,
    -0.011540056,
    0.017296221,
    -0.04856275,
    0.006153595,
    <... 3067 more items ...>,
  ]
)]
result_doc = [ContentEmbedding(
  values=[
    -0.023194725,
    0.021841038,
    0.018162852,
    -0.07353945,
    -0.0050972565,
    <... 3067 more items ...>,
  ]
)]


In [25]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [26]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [27]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

Part 3 - Retrieval

In [28]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [29]:
# Use the newer `invoke` API when available to avoid deprecation warnings
query = "What is Task Decomposition?"
if hasattr(retriever, 'invoke'):
    # `invoke` may return a list of documents or a dict depending on implementation
    try:
        docs = retriever.invoke(query)
    except TypeError:
        # some implementations expect keyword args
        docs = retriever.invoke(texts=[query])
else:
    # Fallback for older langchain versions
    docs = retriever.get_relevant_documents(query)

In [30]:
len(docs)

1